# Predict values using classic machine learning

---

In this notebook, we will choose, train, tune, test, and predict velocity values using a classic machine learning model.

### 1. Load packages, open dataset, load model

Open the AI-ready dataframe that includes the series column for prediction. Load the tuned model from the previous notebook.

In [207]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from pycaret.regression import *

In [208]:
point_name = 'terminus' # 'upstream', 'middle', or 'terminus'
df = pd.read_csv('../data/ai_ready/' + point_name + '_timeseries.csv')

df.head()

,series,time,velocity
0,1,2015-01-30 12:00:00,2033.7096
1,2,2015-02-11 12:00:00,2026.7332
2,3,2015-02-23 12:00:00,2071.6174
3,4,2015-06-11 12:00:00,2092.8370
4,5,2015-06-23 12:00:00,2095.4255


In [209]:
model = load_model('../models/' + point_name + '_model')

Transformation Pipeline and Model Successfully Loaded


In [210]:
df['time'] = pd.to_datetime(df['time'])  # Ensure date is datetime type
df = df.set_index('time').resample('6D').mean(numeric_only=True)  # Resample to every 6 days frequency
df = df.interpolate()  # Interpolate missing values

df.reset_index(inplace=True)
df.head()

,time,series,velocity
0,2015-01-30,1.0,2033.7096
1,2015-02-05,1.5,2030.2214
2,2015-02-11,2.0,2026.7332
3,2015-02-17,2.5,2049.1753
4,2015-02-23,3.0,2071.6174


### 2. Visualize test data

The data were split into training and test data. Here we visualize the predictions for the training data and predictions for the test set (highlighted in light gray).

In [211]:
# Ensure the 'time' column is in datetime format
df['time'] = pd.to_datetime(df['time'])

# Generate predictions and use the original time column
predictions = predict_model(model, data=df)
predictions['time'] = df['time']  # Ensure this matches your data

# Ensure the lengths match
predictions = predictions.iloc[:len(df)]

# Plot
fig = px.line(predictions, x='time', y=["velocity", "prediction_label"])
fig.add_vrect(x0="2021-06-01", x1="2024-04-18", fillcolor="grey", opacity=0.25, line_width=0)
fig.write_image('../figures/test_train' + point_name + '.png')
fig.show()

### 3. Predict future values

The model is now trained and we can predict into the future. Let's start with values 4 years out at a frequency of 6 days.

Here we create a dataset with time and series values, like our original dataframe. The model will fill in the velocity prediction values.

In [212]:
future_dates = pd.date_range(start = '2024-04-18', end = '2028-04-18', freq = '6D')

future_df = pd.DataFrame()

future_df['time'] = future_dates
future_df['series'] = np.arange(145,(145+len(future_dates)))

future_df.head()

,time,series
0,2024-04-18,145
1,2024-04-24,146
2,2024-04-30,147
3,2024-05-06,148
4,2024-05-12,149


Make predictions on the future values.

In [213]:
predictions_future = predict_model(model, data=future_df)
predictions_future.head()

,time,series,prediction_label
0,2024-04-18,145,2275.296106
1,2024-04-24,146,2276.352182
2,2024-04-30,147,2269.204749
3,2024-05-06,148,2281.650062
4,2024-05-12,149,2285.235691


Plot the original data and the future predictions.

In [215]:
concat_df = pd.concat([df,predictions_future], axis=0)
if point_name == 'upstream' or point_name == 'terminus':
    concat_df_i = pd.date_range(start=df['time'].iloc[0], end=predictions_future['time'].iloc[-1] + pd.Timedelta(days=6), freq='6D')
else:
    concat_df_i = pd.date_range(start=df['time'].iloc[0], end=predictions_future['time'].iloc[-1], freq='6D')
concat_df.set_index(concat_df_i, inplace=True)

fig = px.line(concat_df, x=concat_df.index, y=["velocity", "prediction_label"])
fig.write_image('../figures/7_future_preds_' + point_name + '.png')
fig.show()

Overall, the predictions look fairly accurate, with some seasonal spikes during the melt season. The predictions even capture the subtle two spikes in velocity that we have been observing in recent years. Like mentioned earlier, the predictions did not fully capture the intensity of spikes but still were able to capture their general trend.